In [4]:
import pandas as pd

load the dataset

In [5]:
df = pd.read_csv('Sample Dataset.csv')


CHECK FOR BASIC INFOS 


In [7]:
print(df.head()) # display the first few rows of the dataframe
print(df.shape) # display the shape of the dataframe
print(df.info()) # display the summary information of the dataframe
print(df['Price'].describe()) # display the statistical summary of the 'Price' column
print(df.isnull().sum()) # display the count of missing values in each column 

           Brand  Year   Model               Car/Suv  \
0      Ssangyong  2022  Rexton  Sutherland Isuzu Ute   
1             MG  2022     MG3             Hatchback   
2            BMW  2022    430I                 Coupe   
3  Mercedes-Benz  2011    E500                 Coupe   
4        Renault  2022  Arkana                   SUV   

                                       Title UsedOrNew Transmission  \
0       2022 Ssangyong Rexton Ultimate (awd)      DEMO    Automatic   
1  2022 MG MG3 Auto Excite (with Navigation)      USED    Automatic   
2                      2022 BMW 430I M Sport      USED    Automatic   
3           2011 Mercedes-Benz E500 Elegance      USED    Automatic   
4                 2022 Renault Arkana Intens      USED    Automatic   

         Engine DriveType  FuelType FuelConsumption  Kilometres  \
0  4 cyl, 2.2 L       AWD    Diesel  8.7 L / 100 km        5595   
1  4 cyl, 1.5 L     Front   Premium  6.7 L / 100 km          16   
2    4 cyl, 2 L      Rear   Premium

SELECT ONLY USEFUL ONES

In [8]:
features = ['Brand', 'Year', 'Model', 'Car/Suv', 'UsedOrNew', 
            'Transmission', 'Engine', 'DriveType', 'FuelType', 
            'FuelConsumption', 'Kilometres', 'Location', 'CylindersinEngine']

target = 'Price'

CREATE SMALLER DATAFRAMES

In [10]:
df_model = df[features + [target]].copy()
print(df_model.head())
print(df_model.isnull().sum())

           Brand  Year   Model               Car/Suv UsedOrNew Transmission  \
0      Ssangyong  2022  Rexton  Sutherland Isuzu Ute      DEMO    Automatic   
1             MG  2022     MG3             Hatchback      USED    Automatic   
2            BMW  2022    430I                 Coupe      USED    Automatic   
3  Mercedes-Benz  2011    E500                 Coupe      USED    Automatic   
4        Renault  2022  Arkana                   SUV      USED    Automatic   

         Engine DriveType  FuelType FuelConsumption  Kilometres  \
0  4 cyl, 2.2 L       AWD    Diesel  8.7 L / 100 km        5595   
1  4 cyl, 1.5 L     Front   Premium  6.7 L / 100 km          16   
2    4 cyl, 2 L      Rear   Premium  6.6 L / 100 km        8472   
3  8 cyl, 5.5 L      Rear   Premium   11 L / 100 km      136517   
4  4 cyl, 1.3 L     Front  Unleaded    6 L / 100 km        1035   

            Location CylindersinEngine   Price  
0     Caringbah, NSW             4 cyl   51990  
1     Brookvale, NSW    

Handle the Text Data

#ONLY USING THE TEXT 
Already numerical (easy to use):

Year
Kilometres

Text but SIMPLE (few unique values):

Transmission (probably just "Automatic" and "Manual")
UsedOrNew (probably "USED", "NEW", "DEMO")
Car/Suv (probably just "Car" and "SUV")

Text and COMPLEX (many unique values):

Brand (30+ different brands)
Model (100+ different models)
Location (50+ different locations)
Engine (many variations like "4 cyl, 2.0 L")

In [11]:
from sklearn.preprocessing import LabelEncoder

In [13]:
numerical_features = ['Year','Kilometres']
categorical_features = ['Transmission', 'UsedOrNew', 'FuelType', 'DriveType']
target = 'Price'

In [14]:
df_model = df[numerical_features + categorical_features + [target]].copy()

In [15]:
print("Selected features:")
print(df_model.head())
print("\nData types:")
print(df_model.dtypes)

Selected features:
   Year  Kilometres Transmission UsedOrNew  FuelType DriveType   Price
0  2022        5595    Automatic      DEMO    Diesel       AWD   51990
1  2022          16    Automatic      USED   Premium     Front   19990
2  2022        8472    Automatic      USED   Premium      Rear  108988
3  2011      136517    Automatic      USED   Premium      Rear   32990
4  2022        1035    Automatic      USED  Unleaded     Front   34990

Data types:
Year             int64
Kilometres       int64
Transmission    object
UsedOrNew       object
FuelType        object
DriveType       object
Price            int64
dtype: object


In [16]:
print("\nUnique values in categorical features:")
for col in categorical_features:
    print(f"{col}: {df_model[col].unique()}")


Unique values in categorical features:
Transmission: ['Automatic' 'Manual']
UsedOrNew: ['DEMO' 'USED' 'NEW']
FuelType: ['Diesel' 'Premium' 'Unleaded' 'Hybrid' '-']
DriveType: ['AWD' 'Front' 'Rear' 'Other' '4WD']
